In [1]:
%gui qt
import numpy as np
# import cv2
import re

from xvfbwrapper import Xvfb
vdisplay = Xvfb(width=1920, height=1080)
vdisplay.start()
import mayavi.mlab as mlab
mlab.init_notebook()
# trimesh.Scene([scene_mesh]).show()
gripper_width = 0.08
import matplotlib.pyplot as plt
from contact_graspnet.visualization_utils import plot_coordinates, draw_grasps, draw_pc_with_colors, visualize_grasps
from contact_graspnet.data import depth2pc

Notebook initialized with ipy backend.


In [2]:
result_path = '/home/hypatia/6D-DAAD/contact_graspnet/results/predictions_0a00a56361dd4513ba1f9d53967a6b1d.npz'
depth_root = '/home/hypatia/6D-DAAD/GIGA/data/packed/data_packed_train_random_raw_4M_GPG_60_packked/scenes/'
scene_id = re.split('/|_', result_path)[-1]

K = np.array([[540., 0.0, 320.], [0.0, 540., 240.], [0.0, 0.0, 1.0]])
depth_imgs = np.load(depth_root + scene_id, allow_pickle=True)['depth_imgs'][0]
pc_full = depth2pc(depth_imgs, K)[0]


# pc_full[:, 2] *=  -1
results = np.load(result_path, allow_pickle=True)
pred_grasps_cam, scores = results['pred_grasps_cam'].item(), results['scores'].item()

In [3]:
cropped_pc = pc_full[pc_full[:,2]<0.5,:]
fig = visualize_grasps(cropped_pc, pred_grasps_cam, scores, plot_opencv_cam=True, pc_colors=None)

Visualizing...takes time


In [4]:
fig

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…